In [1]:
pip install GPy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.4/959.4 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for GPy: filename=GPy-1.10.0-cp310-cp310-linux_x86_64.whl size=3410106 sha256=a69f020b739c6e74cf925dc852f797b671a588a3f8321823ef4c7c553cc32f39
  Stored in directory: /root/.cache/pip/wheels/27/bd/9f/82ab4216eae088cba864ca0dc1d75699bd4bf6823790fb2f77
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102544 sha256=b56bce9158fb733bee6e27bc4f23d01bd1060aa70146f3fd01b8bb3730df758f
  Stored in directory: /root/.cache/pip/wheels/d2/ef/9c/da9ceef7d0ff5287c24365844fc394852c2b79ac3fcf33bf8b
Successfully built GPy paramz


In [2]:
pip install emukit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.1 MB/s eta 0:00:00
  Created wheel for emukit: filename=emukit-0.4.10-py3-none-any.whl size=257051 sha256=14427054a62a0123f6080c423ac0f36db91adaab2f18a535e2b98b4cc545ad30
  Stored in directory: /root/.cache/pip/wheels/30/d1/af/64ddede19b5eb77bd70362edebf9d393bf77dc34d354c11292
Successfully built emukit


In [3]:
pip install pyDOE

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18168 sha256=84385832543f99bd56e3437f27f653462625528fe3bf6c30f8ae556144eaf7c6
  Stored in directory: /root/.cache/pip/wheels/ce/b6/d7/c6b64746dba6433c593e471e0ac3acf4f36040456d1d160d17
Successfully built pyDOE


In [4]:
import GPy
import numpy as np
from pyDOE import lhs


Read the data

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd


ALL_xdata=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/position_vector_normalized.csv', header=None)
# Full length
# DNS_ydata=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/scaled_wss_dns11data.csv', header=None)

DNS_ydata=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/DNS_data_4_7.csv', header=None)
RANS_ydata=pd.read_csv('/content/drive/MyDrive/ColabNotebooks/RANS_data_4_7.csv',header=None)

Mounted at /content/drive


In [7]:
import numpy as np

# Generate sample data
x_train_l = np.array([[70, 100], [70, 80], [70, 50], [70, 30], [70, 0], [60, 100], [60, 80], [60, 50], [60, 30], [60, 0], [50, 100], [50, 80], [50, 50], [50, 30], [50, 0], [40, 100], [40, 80], [40, 50], [40, 30], [40, 0]])
x_train_h = np.array([[70, 100], [70, 0], [60, 80], [50, 50], [40, 100], [40, 0]])

DNS_full = DNS_ydata.dropna(axis=1)
DNS_full = DNS_full.values
DNS = DNS_full[:, [0, 2, 3, 7, 8, 10]]

DNS_test = DNS_full[:, [1, 4, 5, 6, 9]]
DNS_test = np.transpose(DNS_test)
RANS = RANS_ydata.dropna(axis=1)
RANS_full = RANS.values
RANS_test = RANS_full[:, [0, 1, 3, 4, 5, 6, 7, 10, 12, 13, 14, 15, 16, 17, 19]]

y_train_h = np.transpose(DNS)
y_train_l = np.transpose(RANS)

# Convert y_train_l to a NumPy array
y_train_l = np.array(y_train_l)

# Perform slicing on y_train_l using NumPy indexing


print(np.shape(y_train_h))
print(np.shape(y_train_l))
xdatap = ALL_xdata.values
xdata = xdatap[:, [0, 1]]
print(xdata.shape)
print(len(xdata))
print(DNS_test.shape)


(6, 3600)
(20, 3600)
(3600, 2)
3600
(5, 3600)


In [8]:
print(xdata.shape)

(3600, 2)


Select points within the domain to use GPR to get the full recosnstruction

In [10]:
# from scipy.stats import qmc
# np.random.seed(123)



# # Set the dimension and number of points
# dimension = 1
# num_points =1024
# # Generate Sobol sequence
# sobol_seq = qmc.Sobol(d=dimension, scramble=True).random(num_points)

# # Access the points in the sequence
# points = sobol_seq.T



# # Select indices based on the design points
# selected_indices = np.floor(points * len(xdata)).astype(int)

# Select points from xdata using the selected indices
np.random.seed(123)
num_points =2000# Number of points to select

# Generate Latin Hypercube Design
design = lhs(1, samples=num_points, criterion='maximin')

# Select indices based on the design points
selected_indices = np.floor(design * len(xdata)).astype(int)



[[ 239]
 [ 168]
 [2072]]


High Accuracy (DNS)

In [11]:
# Select corresponding values from ydata
y_train_h =y_train_h[:,selected_indices.flatten()]



Low accuracy (RANS)

In [12]:
y_train_l =y_train_l[:,selected_indices.flatten()]


In [13]:
print(selected_indices)
print(y_train_h.shape)
print(y_train_l.shape)

[[2439]
 [1103]
 [ 547]
 ...
 [2866]
 [2754]
 [2733]]
(6, 2000)
(20, 2000)


In [14]:
# x_train_l=np.c_[ x_train_l, np.zeros(len(x_train_l)) ]
# x_train_h=np.c_[ x_train_h, np.ones(len(x_train_h)) ]
# X_train=np.append(x_train_l,x_train_h,axis=0)
#
from emukit.multi_fidelity.convert_lists_to_array import convert_x_list_to_array, convert_xy_lists_to_arrays

# Y_train=np.append(y_train_h,y_train_l,axis=0)
X_train, Y_train = convert_xy_lists_to_arrays([x_train_l, x_train_h], [y_train_l, y_train_h])
print(X_train.shape)
print(Y_train.shape)

print(X_train)

(26, 3)
(26, 2000)
[[ 70. 100.   0.]
 [ 70.  80.   0.]
 [ 70.  50.   0.]
 [ 70.  30.   0.]
 [ 70.   0.   0.]
 [ 60. 100.   0.]
 [ 60.  80.   0.]
 [ 60.  50.   0.]
 [ 60.  30.   0.]
 [ 60.   0.   0.]
 [ 50. 100.   0.]
 [ 50.  80.   0.]
 [ 50.  50.   0.]
 [ 50.  30.   0.]
 [ 50.   0.   0.]
 [ 40. 100.   0.]
 [ 40.  80.   0.]
 [ 40.  50.   0.]
 [ 40.  30.   0.]
 [ 40.   0.   0.]
 [ 70. 100.   1.]
 [ 70.   0.   1.]
 [ 60.  80.   1.]
 [ 50.  50.   1.]
 [ 40. 100.   1.]
 [ 40.   0.   1.]]


In [15]:
import GPy
import emukit.multi_fidelity
import emukit.test_functions
from emukit.model_wrappers.gpy_model_wrappers import GPyMultiOutputWrapper
from emukit.multi_fidelity.models import GPyLinearMultiFidelityModel
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
%matplotlib inline

In [16]:
kernels = [GPy.kern.Matern32(input_dim=2,lengthscale=[10,10],variance=0.005, ARD=True),GPy.kern.Matern32(input_dim=2,lengthscale=[10,10],variance=0.005, ARD=True)]
# for kernel in kernels:
#     kernel.lengthscale.constrain_bounded(0, 50)
#     kernel.variance.constrain_bounded(0, 10)
for i, kernel in enumerate(kernels):
    if i == 0:  # First fidelity
        kernel.lengthscale.constrain_bounded(0,1000)
        kernel.variance.constrain_bounded(0, 5)
    elif i == 1:  # Second fidelity
        kernel.lengthscale.constrain_bounded(0, 1000)
        kernel.variance.constrain_bounded(0, 10)
lin_mf_kernel = emukit.multi_fidelity.kernels.LinearMultiFidelityKernel(kernels)

gpy_lin_mf_model = GPyLinearMultiFidelityModel(X_train, Y_train, lin_mf_kernel, n_fidelities=2)


gpy_lin_mf_model.mixed_noise.Gaussian_noise_1.fix(0.01)

lin_mf_model =model=GPyMultiOutputWrapper(gpy_lin_mf_model, 2, n_optimization_restarts=200)

## Fit the model

lin_mf_model.optimize()

Optimization restart 1/200, f = -125460.1796624472
Optimization restart 2/200, f = -123410.79772593512
Optimization restart 3/200, f = -123410.75888933893
Optimization restart 4/200, f = -123410.75105499139
Optimization restart 5/200, f = -125460.18280193795


 /usr/local/lib/python3.10/dist-packages/GPy/kern/src/stationary.py:243: RuntimeWarning:invalid value encountered in divide


Optimization restart 6/200, f = -120197.81154503812
Optimization restart 7/200, f = -123410.85416856394
Optimization restart 8/200, f = -123410.75118516512
Optimization restart 9/200, f = -123423.06260378729
Optimization restart 10/200, f = -123423.06257481752
Optimization restart 11/200, f = -120367.04124683874
Optimization restart 12/200, f = -123410.75296437678
Optimization restart 13/200, f = -125460.1828019066
Optimization restart 14/200, f = -123410.88414529155
Optimization restart 15/200, f = -125460.18280160232
Optimization restart 16/200, f = -123410.76340746501
Optimization restart 17/200, f = -123410.75092483014
Optimization restart 18/200, f = -123410.76857497205
Optimization restart 19/200, f = -111492.58380377393
Optimization restart 20/200, f = -125460.18279905237
Optimization restart 21/200, f = -123422.89121173756
Optimization restart 22/200, f = -123279.86889816678
Optimization restart 23/200, f = -125460.18278349823
Optimization restart 24/200, f = -123411.1023939660

In [17]:
 print(gpy_lin_mf_model)


Name : gp
Objective : -125460.18280255086
Number of Parameters : 9
Number of Optimization Parameters : 8
Updates : True
Parameters:
  gp.                                    |                   value  |  constraints  |  priors
  multifidelity.Mat32.variance           |    0.000505037942113845  |    0.0,5.0    |        
  multifidelity.Mat32.lengthscale        |                    (2,)  |  0.0,1000.0   |        
  multifidelity.Mat32_1.variance         |  3.7390734172476537e-16  |   0.0,10.0    |        
  multifidelity.Mat32_1.lengthscale      |                    (2,)  |  0.0,1000.0   |        
  multifidelity.scale                    |      0.1740637015843015  |               |        
  mixed_noise.Gaussian_noise.variance    |  2.6138727375693548e-45  |      +ve      |        
  mixed_noise.Gaussian_noise_1.variance  |                    0.01  |   +ve fixed   |        


In [18]:

#x_plot=np.reshape(x,[10000,2])
x_plot=np.array([[70,50],[60,30],[60,0],[50,80],[40,30]])
X_plot_l=np.c_[ x_plot, np.zeros(len(x_plot)) ]
X_plot_h=np.c_[ x_plot, np.ones(len(x_plot)) ]

In [19]:
hf_mean_lin_mf_model, hf_var_lin_mf_model = lin_mf_model.predict(X_plot_h)


In [20]:
xdatap=ALL_xdata.values
xcp=xdatap[:,[0,1]]
xtdata=xcp[selected_indices.flatten()]
ind_pr=0

In [21]:
predarray=hf_mean_lin_mf_model[ind_pr,:].reshape(-1, 1)



In [22]:


# Create a Gaussian Process Regression model
kernel1 = GPy.kern.ExpQuad(input_dim=2,lengthscale=[0.1,0.1],variance=0.01,ARD=True)
kernel2 = GPy.kern.Matern32(input_dim=2,lengthscale=[0.1,0.1],variance=0.01,ARD=True)
kernel3= GPy.kern.Matern52(input_dim=2,lengthscale=[0.1,0.1],variance=0.01,ARD=True)
kernel=kernel2
# kernel = kernel1*kernel2+kernel2*kernel3+kernel3*kernel1
# kernel = kernel1+kernel1*kernel2*kernel3+kernel2+kernel3+kernel1*kernel2+kernel2*kernel3+kernel1*kernel3
# kernel.lengthscale.constrain_bounded(lower=[0, 0], upper=[10, 10])
# kernel.variance.constrain_bounded(lower=0, upper=1)
gpr_model = GPy.models.GPRegression(xtdata,predarray, kernel)

# gpr_model.Gaussian_noise.fix(0.01)
# Optimize the model parameters  gpr_model.optimize()
gpr_model.optimize_restarts(5)



KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: ignored

In [ ]:
# Predict the entire surface
grid_resolution =200  # Resolution of the grid for predictions
x1 = np.linspace(0,1, grid_resolution)
x2 = np.linspace(0,1, grid_resolution)
X_pred = np.meshgrid(x1, x2)
X_pred = np.vstack((X_pred[0].flatten(), X_pred[1].flatten())).T
Y_pred, Y_pred_var = gpr_model.predict(X_pred)

# Reshape the predictions into a grid
Y_pred_grid = np.reshape(Y_pred, (grid_resolution, grid_resolution))


In [ ]:
# Predict the entire surface
grid_resolution =200  # Resolution of the grid for predictions
x1 = np.linspace(0,1, grid_resolution)
x2 = np.linspace(0,1, grid_resolution)
X_pred = np.meshgrid(x1, x2)
X_pred = np.vstack((X_pred[0].flatten(), X_pred[1].flatten())).T
Y_pred, _ = gpr_model.predict(X_pred)

# Reshape the predictions into a grid
Y_pred_grid = np.reshape(Y_pred, (grid_resolution, grid_resolution))

In [ ]:


# Define the kernel with kernel2
kernel2 = GPy.kern.Matern32(input_dim=2, lengthscale=[0.1, 0.1], variance=0.01, ARD=True)

# Define a list of different initial conditions for hyperparameters
initial_conditions = [
    {"lengthscale": [5, 5], "variance": 0.02},  # Initial condition 1
    {"lengthscale": [0.3, 0.3], "variance": 0.03},  # Initial condition 2
    {"lengthscale": [200,200], "variance": 0.2},  # Initial condition 3
    {"lengthscale": [30,30], "variance": 0.3},  # Initial condition 4
    # Add more initial conditions as needed
]

# Loop over different initial conditions
for i, initial_condition in enumerate(initial_conditions):
    # Create a copy of kernel2 to avoid modifying the original kernel
    kernel = kernel2.copy()

    # Set the initial conditions for hyperparameters
    kernel.lengthscale = initial_condition["lengthscale"]
    kernel.variance = initial_condition["variance"]
    print(kernel.lengthscale)
    print(kernel.variance)
    try:
        # Create the GP regression model with the current kernel
        fullgpr_model = GPy.models.GPRegression(xtdata,predarray, kernel)

        # Optimize the model parameters
        fullgpr_model.optimize_restarts(5)
        # Calculate the negative log-likelihood
        negative_log_likelihood = -fullgpr_model.log_likelihood()
        print(f"Negative Log-Likelihood: {negative_log_likelihood}")
        # Print the optimized parameters and other relevant information
        print(f"Optimization Run {i+1}:")
        print("Optimized Parameters:")
        print(fullgpr_model)
        print("\n")
        # Assuming 'high_gp_model' is your GP regression model
        kernel_params = fullgpr_model.kern.param_array

# # Print kernel parameters
#         print("Lengthscale (Dimension 0):", kernel_params[0])
#         print("Lengthscale (Dimension 1):", kernel_params[1])

# Get the gradients of optimization
        gradients=fullgpr_model.gradient

# Get the names of the model's parameters
        parameter= fullgpr_model.parameter_names()

# Print the gradients along with the parameter names
        print("Gradients of Optimization:")
        for i in range(len(parameter)):
          print(f"Parameter: {parameter[i]} - Gradient: {gradients[i]}")
    except ValueError as e:
        print(f"Error in Optimization Run {i+1}:")
        print(e)


In [ ]:
# # import numpy as np
# # # Y values from -π to π


# # # Save X and Y to CSV files
# # np.savetxt('/content/drive/MyDrive/ColabNotebooks/X_data.csv', X, delimiter=',')
# # np.savetxt('/content/drive/MyDrive/ColabNotebooks/Y_data.csv', Y, delimiter=',')
# np.savetxt('/content/drive/MyDrive/ColabNotebooks/MATmeanL4_7MFsurfE70_50.csv', Y_pred_grid, delimiter=',')
# np.savetxt('/content/drive/MyDrive/ColabNotebooks/MATmeanL4_7RANSGPsurf70_50.csv', rans_pred_grid, delimiter=',')
# np.savetxt('/content/drive/MyDrive/ColabNotebooks/MATmeanL4_7DNSGPsurfE70_50.csv', dns_pred_grid, delimiter=',')
